# `train_test_split()`

In [ ]:
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

Import data, take `TAX`, `CRIM`, and `PTRATIO` and assign to `X`. Assign target to `y`.

In [ ]:
data = load_boston()

print(data.DESCR)

In [ ]:
boston = pd.DataFrame(data.data, columns=data.feature_names)
X = boston[['TAX', 'CRIM', 'PTRATIO']].copy()
boston_y = pd.DataFrame(data.target)

print(X.head())
print(boston_y.head())

Create `X_train, X_test, y_train, y_test` using train-test split with a 33% holdout

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(boston, boston_y, test_size=0.33)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

Fit a linear regression to the **training** set (`X_train` and `y_train`) and look at the $R^2$ score **on the training set**

In [ ]:
lr = LinearRegression()
lr.fit(X_train, y_train)
print(lr.score(X_train, y_train))

Why is the $R^2$ using the **training set** misleading?

**Without refitting the data**, look at the $R^2$ score on the **test set** (`X_test` and `y_test`)

In [ ]:
print(lr.score(X_test, y_test))

Why is this a more accurate measure of how well our linear regression of `TAX`, `CRIM`, and `PTRATIO` on `MDEV`? 

## Try it out!

Import the `datasets/salary.csv` dataset. This dataset records the salary of academics and measures the following features:

- sx = Sex, coded 1 for female and 0 for male
- rk = Rank, coded
  - 1 for assistant professor,
  - 2 for associate professor, and
  - 3 for full professor
- yr = Number of years in current rank
- dg = Highest degree, coded 1 if doctorate, 0 if masters
- yd = Number of years since highest degree was earned
- sl = Academic year salary, in dollars.

Import the salary data, assign the `yr` and `yd` columns to a variable named `X`, then assign `sl` to a variable named `y`

Use `train_test_split()` to create a training set and a test set, split 50/50

Train a linear regression using the training set. What does the $R^2$ look like for the training set? What does the $R^2$ look like for the test set? Which is higher? Which is a more accurate number? Why?

# `cross_val_score()` and `cross_val_predict()`

Let's use `cross_val_score()` and `cross_val_predict()` to do cross-validation and to take a quick look at each fold.

Using the boston data, let's pick `CRIM`, `ZN`, and `INDUS` for our predictors and assign them to `X`

In [ ]:
X = boston[['CRIM', 'ZN', 'INDUS']].copy()
print(X.head())
print(boston_y.head())

Use `cross_val_score()` and five-fold cross-validation to see the $R^2$ for each of the folds compared to the test set for that fold:

In [ ]:
from sklearn.model_selection import cross_val_score, cross_val_predict

lr = LinearRegression()

print(cross_val_score(lr, X, boston_y, cv=5))

What do the low $R^2$ scores mean? If they were there, what does a **negative** $R^2$ show?

We may also want to see an overall score for these folds. Typically, we will take the average $R^2$ across each fold:

In [ ]:
scores = cross_val_score(lr, X, boston_y, cv=5)
print(scores.mean())

Use `cross_val_predict()` and assign the predicted values for each test fold to an object called `predictions`. How many predictions should we have in total?

In [ ]:
predictions = cross_val_predict(lr, X, boston_y, cv=5)

`cross_val_predict()` lets us use the predictions from each fold to look at metrics that are important but not part of `LinearRegression()`'s `.score()` method, such as the mean squared error. The mean squared error is the average difference between the true y and the predicted y:

$$ \text{Mean Squared Error} = \frac{\sum(y_i - \hat{y}_i)^2}{n} $$

We will import sklearn's `mean_squared_error()` function which takes in a true y and a predicted y and outputs the MSE

In [ ]:
from sklearn.metrics import mean_squared_error

mean_squared_error(boston_y, predictions)

## Try it out!

Using the Boston housing data above, try a different set of columns -- does the $R^2$ on the folds shown by `cross_val_score()` improve? What about the MSE using the predictions output by `cross_val_predict()`?

# `KFold`

Sklearn also lets us manually create folds, which we can then call other options on. This is helpful if we want to do more advanced work on each of those folds. 

In the following example, we will fit a linear regression using every column in the Boston housing data and validating its efficacy using 5-fold cross-validation. For each fold, I will calculate the $R^2$ and the MSE -- this would be difficult to do without using `KFolds` to manually split up the data.

For ease of indexing, I am converting `X` into a `numpy` array using the `.values` attribute

In [ ]:
from sklearn.model_selection import KFold

In [ ]:
X = boston.copy().values
boston_y = boston_y.values

print(X.shape, boston_y.shape)

Instantiate a `KFold` object with 5 folds

In [ ]:
kfolds = KFold(5)

`KFold` will take in one or two data collections and return back a set of paired indices for the folds in question:

In [ ]:
for training_index, test_index in kfolds.split(X, boston_y):
    print('training: ', training_index[0:5], '...', training_index[-6:-1], 
          '\ntest: ', test_index[0:5], '...', test_index[-6:-1], '\n')

We will use these indices to manually split up `X` and `boston_y`:

In [ ]:
counter = 0
for training_index, test_index in kfolds.split(X, boston_y):
    x_train = X[training_index]
    x_test = X[test_index]
    y_train = boston_y[training_index]
    y_test = boston_y[test_index]
    print('fold', counter, '-', x_train.shape, x_test.shape, y_train.shape, y_test.shape)
    counter += 1 

We then feed these folds into `LinearRegression()` to fit it to a given fold and then use `.score()` and `mean_squared_error()` to look at the fit on the test set in each fold:

In [ ]:
counter = 0
lr = LinearRegression()
for training_index, test_index in kfolds.split(X, boston_y):
    x_train = X[training_index]
    x_test = X[test_index]
    y_train = boston_y[training_index]
    y_test = boston_y[test_index]
    lr.fit(x_train, y_train)
    r2_score = lr.score(x_test, y_test)
    fold_predictions = lr.predict(x_test)
    mse_score = mean_squared_error(y_test, fold_predictions)
    print('Fold', counter, '- r^2 score:', r2_score, 'mse:', mse_score)
    counter += 1

By using `KFold` we are able to access the folds _as they are created_ and can learn more about how well each fold does using different statistics. 

**Discussion Questions**:

1. Why might we want to look at different goodness of fit metrics during the cross-validation process?
2. It looks like our $R^2$ changes pretty frequently but our `mse` is not particularly volatile in this process. Have we made a good modeling choice by including every column in Boston? Why or why not? 